In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Se carga el set de entrenamiento

In [3]:
prop = pickle.load(open("../../setEntrenamiento.p","rb"))

In [4]:
#cortamos por precios
prop = prop[(prop['price_usd_per_m2']<9062)]
prop.reset_index(inplace=True)

p = prop[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','price_aprox_usd']]

### Se le asigna un valor numerico al tipo de propiedad

In [5]:
def asignarPT( p ):
    if p == 'apartment':
        return 3
    if p == 'house':
        return 2.25
    if p == 'store':
        return 6.5
    return 2.5

In [6]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Se le asigna un valor a los barrios segun el analisis de grupos hecho en el tp1

In [7]:
def asignarPV(name, dic):
    return dic[name]   

In [8]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [9]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [10]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
pEnt = p[['property_type','surface_total_in_m2',\
    'place_value', 'price_aprox_usd']]
pEnt.dropna(inplace=True)

/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Se carga el set de Prueba

In [12]:
proper = pickle.load(open("../../setPrueba.p","rb"))

In [13]:
#se formatea lat y lon
p = proper[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

p['lat'] = p.apply(lambda row: row[4][0] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p['lon'] = p.apply(lambda row: row[4][1] if(np.all(pd.notnull(row[5]))) else row[5], axis=1)

p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

def asignarPVsegunEnt(name):
    if(HGdics.has_key(name)):
        return HGdics[name]
    return np.nan

#hay que armar los hoodGropus segun los resultados del set de Entrenamiento
p["place_value"] = p.apply(lambda x: asignarPVsegunEnt(x[1]), axis=1)

pPru = p[['property_type','surface_total_in_m2',\
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'place_name']]

/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/rodrigoderosa/.local/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


### Prediccion del precio

In [14]:
#definir el modelo a probar
model = GradientBoostingRegressor(loss = 'ls', n_estimators = 5000, max_depth = 5, learning_rate = 0.241)

In [15]:
#fit model on training dataset
model.fit(pEnt[['property_type','surface_total_in_m2','place_value']], pEnt['price_aprox_usd'])

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.241, loss='ls', max_depth=5,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=5000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [16]:
#prediccion 
pred = mean_squared_error(pEnt['price_aprox_usd'], model.predict(pEnt[['property_type','surface_total_in_m2',\
    'place_value']]))
print ('RMSE_Price', pred)

('RMSE_Price', 6909913267.7700148)


In [17]:
from sklearn.preprocessing import Imputer
#hay que llenar el resto de los valores nan

# Create our imputer to replace missing values with the mean e.g.
imp = Imputer(missing_values=np.nan, strategy='most_frequent', axis=1)
imp = imp.fit(pEnt)

pPru_imp = imp.transform(pPru[['property_type','surface_total_in_m2',
        'place_value']])

In [20]:
pPru['price_usd'] = model.predict(pPru_imp)
result = pd.DataFrame()
result["id"] = proper["id"]
result["price_usd"] = pPru['price_usd'].apply(lambda x: -x if x < 0 else x)
result.describe()

,id,price_usd
count,1.416600e+04,1.416600e+04
mean,2.285382e+06,2.023755e+05
std,4.290968e+04,2.263596e+05
min,3.632000e+03,1.242040e+04
25%,2.271809e+06,1.001261e+05
50%,2.290284e+06,1.381702e+05
75%,2.309395e+06,2.143010e+05
max,2.332338e+06,6.194956e+06


In [21]:
result.to_csv(index=False, path_or_buf="resultsFinalPhiAlpha.csv")